# Action Designator
This example will show the different kinds of Action Designator that are available. We will see how to create Action Designators and what they do.


Action Designator are high-level descriptions of actions which the robot should execute. 

Action Deisgnators are created from a Action Designator Descritpion, which describes the type of action as well as the parameter for this action. Parameter are given as a list of possible parameters.
For example, if you want to describe the robot moving to a table you would need a ```NavigateAction``` and a list of poses that are near the table. The Action Designator Description will then pick one of the poses and return a performable Action Designator which contains the picked pose. 



## Navigate Action
We will start with a simple example of the ```NavigateAction```. 

First, we need a BulletWorld with a robot.

In [2]:
from pycram.bullet_world import BulletWorld, Object

world = BulletWorld()
pr2 = Object("pr2", "pr2", "pr2.urdf")

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


In [28]:
world.exit()

To move the robot we need to create a description and resolve it to an actual Designator. The description of navigation only needs a list of possible poses.

In [3]:
from pycram.designators.action_designator import NavigateAction

pose = [[1, 0, 0], [0, 0, 0, 1]]

# This is the Designator Description
navigate_description = NavigateAction(target_locations=[pose])

# This is the performable Designator
navigate_designator = navigate_description.resolve()

What we now did was create the pose were we want to move the robot, create a description describing a navigation with a list of possible poses (in this case the list contains only one pose) and create an action designator from the description. The action designator contains the pose picked from the list of possible poses and can be performed.

In [4]:
from pycram.process_module import simulated_robot

with simulated_robot:
    navigate_designator.perform()

Every designator that is performed needs to be in an environment that specifies where to perform the designator either on the real robot or the simulated one. This environment is called ```simulated_robot``` similar there is also a ```real_robot``` environment. 

There are also decorators which do the same thing but for whole methods, they are called ```with_real_robot``` and ```with_simulated_robor```

## Move Torso
This action designator moves the torso up or down, specifically it sets the torso joint to a given value.

We start again by creating a description and resolving it to a designator. Afterwards, the designator is perfomed in a ```simulated_robot``` environment. 

In [6]:
from pycram.designators.action_designator import MoveTorsoAction
from pycram.process_module import simulated_robot

torso_pose = 0.2

torso_desig = MoveTorsoAction([torso_pose]).resolve()

with simulated_robot:
    torso_desig.perform()

## Set Gripper
As the name implies, this action designator is used to open or close the gripper. 

The procedure is similar to the last time, but this time we will shorten it a bit.

In [7]:
from pycram.designators.action_designator import SetGripperAction
from pycram.process_module import simulated_robot

gripper = "right"
motion = "open"

with simulated_robot:
    SetGripperAction(grippers=[gripper], motions=[motion]).resolve().perform()

## Park Arms
Park arms is used to move one or both arms into the default parking position.

In [8]:
from pycram.designators.action_designator import ParkArmsAction
from pycram.process_module import simulated_robot
from pycram.enums import Arms

with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()

## Pick Up and Place
Since these are depending on each other, meaning you can only place something when you picked it up beforehand, they will be shown together. 

These action designators use object designators, which will not be further explained so please check the example on object designators for more details. 

To start we need an environment in which we can pick up and place things as well as an object to pick up.

In [9]:
kitchen = Object("kitchen", "environment", "kitchen.urdf")
milk = Object("milk", "milk", "milk.stl", position=[1.3, 1, 0.9])

world.reset_bullet_world()

In [14]:
from pycram.designators.action_designator import PickUpAction, PlaceAction, ParkArmsAction, MoveTorsoAction, NavigateAction
from pycram.designators.object_designator import BelieveObject
from pycram.process_module import simulated_robot
from pycram.enums import Arms

milk_desig = BelieveObject(names=["milk"])
arm ="right"

with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    
    MoveTorsoAction([0.3]).resolve().perform()
    
    NavigateAction([[[0.72, 0.98, 0.0], 
                     [0.0, 0.0, 0.014701099828940344, 0.9998919329926708]]]).resolve().perform()
    
    PickUpAction(object_designator_description=milk_desig, 
                     arms=[arm], 
                     grasps=["right"]).resolve().perform()
    
    NavigateAction([[[-1.90, 0.78, 0.0], 
                     [0.0, 0.0, 0.16439898301071468, 0.9863939245479175]]]).resolve().perform()
    
    PlaceAction(object_designator_description=milk_desig, 
                target_locations=[[[-1.20, 1.0192, 0.9624], 
                                   [0.0, 0.0, 0.6339889056055381, 0.7733421413379024]]], 
                arms=[arm]).resolve().perform()

In [15]:
world.reset_bullet_world()

## Look At
Look at lets the robot look at a specific point, for example if it should look at an object for detecting. 

In [17]:
from pycram.designators.action_designator import LookAtAction
from pycram.process_module import simulated_robot

target_location = [[1, 0, 0.5], [0, 0, 0, 1]]
with simulated_robot:
    LookAtAction(targets=[target_location]).resolve().perform()

# Detect
Detect is used to detect objects in the field of vision (FOV) of the robot. We will use the milk used in the pick up/place example, if you didn't execute that example you can spawn the milk with the following cell. The detect designator will return a resolved instance of an ObjectDesignatorDescription. 

In [ ]:
milk = Object("milk", "milk", "milk.stl", position=[1.3, 1, 0.9])

In [27]:
from pycram.designators.action_designator import DetectAction, LookAtAction, ParkArmsAction
from pycram.process_module import simulated_robot

milk_desig = BelieveObject(names=["milk"])
print(milk_desig.resolve().pose)

with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    
    NavigateAction([[[0, 1, 0], [0, 0, 0, 1]]]).resolve().perform()
    
    LookAtAction(targets=[milk_desig.resolve().pose]).resolve().perform()
    
    obj_desig = DetectAction(milk_desig).resolve().perform()

((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))


PerceptionObjectNotFound: Could not find an object with the type milk in the FOV of the robot